In [1]:
from scipy.special import expit #to calculate logit 
import scipy.io as si
import numpy as np

#.mat file has two matrix X (5000*400) and y(5000*1)
File = si.loadmat('ex4data1.mat')

#sigmoid function implementation
def sigmoid_function(z):
          return expit(z)

#sigmoid gradient implementation
def sigmoid_gradient(Z):
    temp = sigmoid_function(Z)
    return (temp*(1 - temp))

#implement Forward Propagation
def compute_input_layer(theta1,input_layer):
    return (sigmoid_function(np.dot(input_layer,theta1.T)))

def compute_hidden_layer(theta2,A):
    return (sigmoid_function(np.dot(A,theta2.T)))

#return 5000*10 matrix
def compute_hypothesis(Theta1,Theta2,input_layer):
    
    #get 5000*25 matrix
    A1 = compute_input_layer(Theta1,input_layer)
    
    #change A dim to 5000*26
    A1 = np.insert(A1,0,1,axis = 1)

    #get 5000*10 matrix
    A2 = compute_hidden_layer(Theta2,A1)
    return A2 , A1

#implement Cost function for Neural Nertwork with no Regularization

#return the actual result of Kth classifier for a given data in training set
def get_result(actual_result,K,data):
    if actual_result[data] == K:
        return 1
    else:
        return 0

#return the value for Kth classifier for a given data in training set
def get_hypo_value_K(hypothesis,data,K):
        return (hypothesis[data].item(K))

#calculate cost for a given training data and parameters
def Cost_function_n_reg(theta1,theta2,input_layer,actual_result,K):
    
    #calculate hypothesis
    hypothesis1 , hypothesis2 = compute_hypothesis(theta1,theta2,input_layer)
    
    cost = 0
    #iterate it for each training data
    for i in range(len(hypothesis1)):
        #for each class
        for j in range(K):
            term1 =  - get_result(actual_result,j,i)*np.log(get_hypo_value_K(hypothesis1,i,j)) 
            term2 =  - (1 - get_result(actual_result,j,i))*np.log(1 - get_hypo_value_K(hypothesis1,i,j)) 
            cost = cost + term1 + term2
            
    cost = (cost/(len(hypothesis1)))
    return cost


In [2]:
print(sigmoid_gradient(np.array([-1, -0.5 ,0, 0.5, 1])))

[ 0.19661193  0.23500371  0.25        0.23500371  0.19661193]


In [3]:
#load input_layer
input_layer = np.array(File['X'])
input_layer = np.insert(input_layer,0,1,axis = 1)

#randomly initializes theata1(25,401) and theta2(10,26) for a given epsilon


epsilon = 0.12
init_theta1 = np.random.uniform(-epsilon,epsilon,(25,401))
init_theta2 = np.random.uniform(-epsilon,epsilon,(10,26))

In [4]:
print(init_theta1.shape)

(25, 401)


In [5]:
#create actual result of training set in form of {0,1}
def result_binary(actual_result,num_labels):
    y = np.zeros((len(actual_result),num_labels))
    for i in range(len(actual_result)):
        temp = actual_result[i]
        y[i][temp - 1] = 1
    return y

output_layer = result_binary(File['y'],10)
print(output_layer[4999])

[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]


In [6]:
hypothesis1 , hypothesis2 = compute_hypothesis(init_theta1,init_theta2,input_layer)
delta_3 = hypothesis1 - output_layer
print(delta_3.shape)

(5000, 10)


In [7]:
term1 = np.dot(delta_3,init_theta2).T
term1 = np.delete(term1,0,0)
delta_2 = np.multiply(term1.T,sigmoid_gradient(np.dot(input_layer,init_theta1.T)))

print(delta_2.shape)

(5000, 25)


In [8]:
grad_1 = np.dot(input_layer.T,delta_2).T
grad_1.shape

(25, 401)

In [9]:
print(hypothesis2.shape)
print(delta_3.shape)
grad_2 = np.dot(hypothesis2.T,delta_3)
grad_2 = np.delete(grad_2,0,0).T
print(grad_2.shape)

(5000, 26)
(5000, 10)
(10, 25)


In [10]:
#implement BackPropagation
def BackPropagation(init_theta1,init_theta2,input_layer,output_layer):
    
    #calculate forward propagation
    hypothesis1 , hypothesis2 = compute_hypothesis(init_theta1,init_theta2,input_layer)
    
    #computes small delta
    
    #compute delta_3
    delta_3 = hypothesis1 - output_layer
    
    #compute delta_2
    term1 = np.dot(delta_3,init_theta2).T
    term1 = np.delete(term1,0,0)
    delta_2 = np.multiply(term1.T,sigmoid_gradient(np.dot(input_layer,init_theta1.T)))
    
    #compute gradient for layer 1
    grad_1 = np.dot(input_layer.T,delta_2).T
    grad_1 = grad_1/len(hypothesis1)
    
    #compute gradient for layer 2
    grad_2 = np.dot(hypothesis2.T,delta_3)
    grad_2 = np.delete(grad_2,0,0).T
    grad_2 = grad_2/len(hypothesis2)
    
    return (grad_1 , grad_2)

GRADIENT_1 , GRADIENT_2 = BackPropagation(init_theta1,init_theta2,input_layer,output_layer)

In [11]:
print(GRADIENT_1.shape,GRADIENT_2.shape)

(25, 401) (10, 25)


In [12]:
#perform  Gradient Checking on a small neural network

#make a small neural network

#define sizes
Lambda = 0
input_layer_size = 3 #including bias unit
hidden_layer_size = 5 #including bias unit
num_labels = 3
m = 5

#put some random values
#randomly  initializes test_theta's with a range of (-.12,.12)
epsilon = 0.12
test_theta1 = np.random.uniform(-epsilon,epsilon,(hidden_layer_size -1,input_layer_size ))
test_theta2 = np.random.uniform(-epsilon,epsilon,(num_labels,hidden_layer_size ))

#randomly generate input layer (5*2)
test_input_layer = np.random.rand(m,input_layer_size - 1)
test_input_layer = np.insert(test_input_layer,0,1,axis = 1)
print(test_input_layer)

#randomly generate  actual result 
test_result = np.random.randint(0,3,(m,1))
print(test_result)

#create test actualresult in binary form
test_actual_result = np.zeros((len(test_result),num_labels))
for i in range(len(test_result)):
         test_actual_result[i][test_result[i]] = 1

print(test_actual_result)

#make a long test_THETA VECTOR 
test_theta_L = np.concatenate((test_theta1.flatten(),test_theta2.flatten()))
print(test_theta1.shape)
print(test_theta2.shape)
print(test_theta_L)

[[ 1.          0.37472727  0.94687881]
 [ 1.          0.98595169  0.9216116 ]
 [ 1.          0.77952787  0.40017177]
 [ 1.          0.44324102  0.17411526]
 [ 1.          0.78086834  0.88055597]]
[[1]
 [0]
 [0]
 [0]
 [0]]
[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]]
(4, 3)
(3, 5)
[-0.11088972 -0.03380308 -0.07400609 -0.09408612 -0.04424503  0.09828427
  0.06051384 -0.11566431  0.03934134 -0.01766719 -0.03754778 -0.00017925
  0.05500626  0.00336233 -0.07651638  0.01897128  0.02481286 -0.05106702
 -0.04654575  0.10761832  0.10874479  0.0407346   0.0454665  -0.03393324
  0.00912121  0.06582694  0.11768603]


In [13]:
#make calls to backpropagation and Cost_function for small neural network
TEST_GRADIENT_1 , TEST_GRADIENT_2 = BackPropagation(test_theta1,test_theta2,test_input_layer,test_actual_result)

In [14]:
print(Cost_function_n_reg(test_theta1,test_theta2,test_input_layer,test_result,3))

2.14796640752


In [15]:
def gradient_checking(test_theta_L,epsilon):
    GRADIENT_CHECK = np.zeros((len(test_theta_L),1))
    for i in range(len(test_theta_L)):
        inc = np.zeros((len(test_theta_L),1))
        inc[i] = inc[i] + epsilon
    
        theta_plus = test_theta_L + inc
        theta_minus = test_theta_L - inc
    
        #unrolling for theta_plus
        pass_theta1 = np.array(theta_plus[0:12]).reshape(4,3)
        pass_theta2 = np.array(theta_plus[12:]).reshape(3,5)
    
        cost_plus = Cost_function_n_reg(pass_theta1,pass_theta2,test_input_layer,test_result,3)
    
        #unrolling for theta_minus
        pass_theta1 = np.array(theta_minus[0:12]).reshape(4,3)
        pass_theta2 = np.array(theta_minus[12:]).reshape(3,5)
    
        cost_minus = Cost_function_n_reg(pass_theta1,pass_theta2,test_input_layer,test_result,3)
    
        GRADIENT_CHECK[i] = (cost_plus - cost_minus)/(2*epsilon)
    
    return GRADIENT_CHECK


In [16]:
test_theta_L = test_theta_L.reshape(len(test_theta1.flatten()) + len(test_theta2.flatten()) ,1)
GRADIENT_CHECK = gradient_checking(test_theta_L,.0001)
#COMPARE OUTPUT OF GRADIENT_CHECKING AND BACKPROPAGATION

GRADIENT_CHECK1 = np.array(GRADIENT_CHECK[0:12]).reshape(4,3)
GRADIENT_CHECK2 = np.array(GRADIENT_CHECK[12:]).reshape(3,5)
GRADIENT_CHECK2 = np.delete(GRADIENT_CHECK2,0,1)

print("output from Backpropagtion for theta 1 \n")
print(TEST_GRADIENT_1)
print("\noutput from Gradient Checking for theta 1 \n")
print(GRADIENT_CHECK1)

print("\noutput from Backpropagtion for theta 2 \n")
print(TEST_GRADIENT_2)
print("\noutput from Gradient Checking for theta 2 \n")
print(GRADIENT_CHECK2)

output from Backpropagtion for theta 1 

[[-0.00833061 -0.00633859 -0.00483304]
 [ 0.01516642  0.01294939  0.0074928 ]
 [ 0.01589434  0.01203138  0.00930053]
 [ 0.01701806  0.01168611  0.01108773]]

output from Gradient Checking for theta 1 

[[-0.00833061 -0.00633859 -0.00483304]
 [ 0.01516642  0.01294939  0.0074928 ]
 [ 0.01589434  0.01203138  0.00930053]
 [ 0.01701806  0.01168611  0.01108773]]

output from Backpropagtion for theta 2 

[[-0.13220065 -0.13858658 -0.14322924 -0.14119313]
 [ 0.14306104  0.15018876  0.15523379  0.15288642]
 [ 0.24140897  0.25784969  0.26679253  0.25990196]]

output from Gradient Checking for theta 2 

[[-0.13220065 -0.13858658 -0.14322924 -0.14119313]
 [ 0.14306104  0.15018876  0.15523379  0.15288642]
 [ 0.24140897  0.25784969  0.26679253  0.25990196]]


In [ ]:
#learn parameters 